# ユニバーサル Abacus マシン

数理論理学 B （立木担当）のレポート課題です。

このファイル(mathlogic.ipynb)だけをダウンロードすることもできますが，このフォルダー(プログラミング演習（数理的応用）という，python の演習授業のオンラインテキスト)全体をダウンロードすることをおすすめします。その方法は，この上のBranch: master python/mathlogic.ipynb と書かれている python の部分をクリックしてください。

メディアセンタのパソコンには，github および anaconda （python に，よく使われるパッケージを最初からインストールしたもの）がインストールされています。これは，他にも使う機会があると思うので，これを機会に自分のパソコンにインストールすることをおすすめします。このフォルダの README.md を見てください。github はなくても，このファイルをダウンロードできれば課題を実行できます。

Google アカウントを持っている人は，
<a href="https://colab.research.google.com/notebooks/welcome.ipynb?hl=ja">
Google Colaboratory </a>を用いると，anaconda などを自分のコンピュータに
インストールをせずに Jupyter Notebook で書かれた Python プログラムを動作できます。
上でリンクされている Coogle Colaboratory のページの "GitHub でのノートブックの保存と読み込み" のリンクの先に，Github で公開された
ページを Google Colaboratory で開く方法が載っています。具体的には，
<a href="https://colab.research.google.com/github/Hidekitsuiki/python/blob/master/mathlogic.ipynb">このリンク</a>
によって，上記課題ページを Google Colaboratory で実行できます。
ただし，Google Colaboratory の方が再帰呼び出しの深さの制限がきついので，動かせる Abacus マシンプログラムのサイズは
より小さくなります。Google colaboratory の利用は，個人の責任で行なってください。

この notebook ファイルの最後の練習問題１，２を行ってください。
練習問題１は必ず行ってください。練習問題 2 は任意です。練習問題 3 は，自分で考えるだけで十分で，提出する必要はありません。

この notebook にセルを追加して，プログラムを書いて実行したり，説明の文章をMarkdown で書いたりしてください。そして，できた notebook の名前を，"自分の名前.ipynb"　に変えて，PandA の第2回という課題に添付として提出してください。

この課題を行うのに，python のプログラムを理解する必要はありません。ただ，Jupyter Notebook の利用には慣れている必要があります。必要に応じて，このフォルダにある Pythonのオンラインテキスト，特に，1Jupyter.ipynb を参照してください。

準備のところのプログラムは，理解する必要はありません。Ctrl-return によりプログラムを実行しながら読み進めてください。

## (準備)2分探索による逆関数

自然数上の単調関数 $f$ を計算するプログラムに対し，b 以下の範囲で，p <= f(n) となる最小の n を求める関数です。
2分探索を行っています。これは，下で numpair 関数のプログラムを書くのに用います。

In [1]:
# 練習問題40

def bsearch (p, f, b):
    "For monotonic f, binary search least n such that p <= f(n)  up to b"
    if (f(b) < p):
        return b   # this means that no such n exists
    return bsearchrange (p, f, 0, b)

def bsearchrange (p, f, a, b):
    if(b <= a+1):
        return b
    c = (a+b)//2
    if (f(c) < p):
        return (bsearchrange (p, f, c, b))
    else:
        return (bsearchrange (p, f, a, c))
    
bsearch (36, lambda n:n*n, 100)    


6

## (準備)ペアのコーディング

ここでは，0 以上の数を自然数ということにします。自然数全体の集合$\mathbb N$ と，そのペア全体の集合$\mathbb N \times \mathbb N$ は1対1対応をつけることができます(カントールの対関数)。ペア全体に
三角形に番号をつける，つまり (0,0), (1,0), (0,1), (2.0), (1,1), (0,2), (3.0), ... と番号をつけていきます。
この1対1対応を作る $\mathbb N \times \mathbb N \to\mathbb N$の関数 pairnum(x, y) と，その逆関数 numpair(z)
$\mathbb N \to\mathbb N \times\mathbb N$ のプログラムです。numpair は，bsearch を用いて実現されており，タプルを返しています。


In [2]:

import math
def pairnum(x, y):
    return (x + y)* (x + y + 1)//2 + y

def numpair(z):
    kk = bsearch(z, lambda i:pairnum(i, 0)-1, z+1) - 1

    r = pairnum(kk, 0)
    y = z - r
    x = (kk) - y
    return (x, y)


# [pairnum(*numpair(i))==i for i in range(100)]


## (準備)リストのコーディング

自然数の集合$\mathbb N$ と，自然数の有限列(すなわちリスト)全体の集合$\mathbb N^*$ との間で，次のようにして，
1対1対応をつけることができます。
$$
\text{listnum}(u) = \left\{\begin{array}{ll} 0 & (u = []) \\ \text{pairnum}(u[0], \text{listnum}(u[1:])+1 & (それ以外)\end{array}\right.
$$
この対応とその逆対応( numlist(z) ) のプログラムです。ここでは，再帰を用いています。後に述べる universal マシンを Abacus マシンにコンパイルするためには，再帰を用いずにプログラムを書く必要がありますが，それについては，授業中に配布したプリントを参照してください。


In [3]:
# 練習問題60

def listnum(u):  #u:list
    if (u==[]):
        return 0
    else:
        return pairnum(u[0], listnum(u[1:]))+ 1

def numlist(n):
    if(n == 0):
        return []
    else:
        (a, b) = numpair(n - 1)
        return([a] + numlist(b))

    
#k = listnum([1,2,3])
#k = listnum([1,1,1,1,1,1,1,1,1,1,1,1])
#print(k)
#print(numlist(k))


ところで、このプログラムでは、ここら辺が限界で、もっと大きなリストをエンコードしようとしても、bsearchのところでの再帰呼び出しが深くなりすぎて、エラーを生じてしまいます。
numpair に関しては、次が知られています。
```
# (wikipedia 「対関数」参照)
import math
def numpair(z):
    w = (int)((math.sqrt(8*z+1)-1)//2)
    t = (w*w+w)//2
    y = z-t
    x = w-y
    return(x,y)
```
これで行えば2分探索の必要はなくなるが、今度は、math.sqrt が, 浮動小数点で扱えるより大きな引数が与えられてエラーが発生します。(その前に、大きな数を浮動小数点にした時に誤差が生じる。）それを防ぐには、整数範囲内でのルート計算を行わないといけなくなります。それは、例えば，ニュートン法でできるはずです。



## (準備)コード化されたリストへのアクセス

nth : 数 x でコード化されたリストの k 番目の要素を取り出す関数，

replace : 数 x でコード化されたリストの k 番目の要素を n に置き換えたリストのコードを返す関数

In [4]:
def nth(x, k):
    if(x == 0):
        return 0  # 長さを超えたアクセスは 0 を返す
    (a, y) = numpair(x-1)
    if(k == 0):
        return a
    else: 
        return nth(y, k-1)

def replace(x, k, n):   # x[k] = n
    if(x == 0):
        if (k == 0): 
            a = n
            rest = 0
        else:
            a = 0
            rest = replace(0, k-1, n)
        return pairnum(a, rest) + 1
    (a, y) = numpair(x-1)
    if(k == 0):
        return pairnum(n, y)+ 1
    else:
        return pairnum(a, replace(y, k-1, n))+ 1

z = listnum([1,2,3,4,5])    
print(nth(z, 5))
u = replace(z, 4, 8)     
print(numlist(u))

z = listnum([1,2,3,4,5])
print(nth(z, 10))
u = replace(z, 9, 8)
print(numlist(u))
        

0
[1, 2, 3, 4, 8]
0
[1, 2, 3, 4, 5, 0, 0, 0, 0, 8]


# ユニバーサル Abacus マシン(その1)

授業プリントと同じプログラムを python で書いたものです。
プリントでは p, d として自然数を与えているが，ここでは python のリストで与えています。よって，リストの長さより先にアクセスできないので，データ d としては，プログラム中に現れる箱の番号を全て含むように，十分長く 0 を後ろにつけたものを与える必要があります。


In [5]:
def univ(p, d):
    pc = 0
    while(p[pc][0] > 0):
        op = p[pc][0]
        box = p[pc][1]
        next = p[pc][2]
        if(op == 1):
            d[box] += 1
            pc = next
        else:
            enext = p[pc][3]
            if(d[box] > 0):
                d[box] -= 1
                pc = next
            else:
                pc = enext
    return(d[0])

# 様々な関数の Abacus マシンでのコード

例１：何もしないでいきなり終了する. どの k に対しても f(x1,...,xk) = 0 を意味する。


In [6]:
m = [[0]]
d = [0,1,6,4,3]   # 引数としては，任意のリスト。d[0] は返値，d[1,...,k] が引数
univ(m,d)

0

例２：箱1 の内容を箱0 にコピーする。箱0は空にする。 f (x) = y を意味する。

In [7]:
d = [0, 5]
m = [[2,1,1,2],
 [1,0,0],
 [0]
]  
univ(m, d)

5

例３：箱 0 の内容を B[1] + B[2] にする。箱 1, 2 は終了時に空になる。
f(x, y) = x+y を意味する。

![足し算](plus_machine.jpg)

In [8]:
d = [0, 5, 3]
m = [[2,1,1,2],
 [1,0,0],
 [2,2,3,4],
 [1,0,2],
 [0]
]  
univ(m, d)

8

# ユニバーサル Abacus マシン(その2)

nth, replace を用いて，上の univ のプログラムにおいて，プログラム p とデータ d を，リストではなく数を引数とするように書きかえたもの。

In [9]:
def univm(p, d):  
    ''' p, d は，リストを数で表した数が与えられる '''

    pc = 0
    while(nth(nth(p, pc), 0) > 0):
        print(pc, numlist(d))  # デバッグのためのプリント
        op = nth(nth(p, pc), 0)
        box = nth(nth(p, pc), 1)
        next = nth(nth(p, pc), 2)
        if(op == 1):
            d = replace(d, box, nth(d, box)+1)
            pc = next
        else:
            enext = nth(nth(p, pc), 3)
            if(nth(d,box) > 0):
                if(nth(d, box) > 0):
                    d = replace(d, box, nth(d, box) - 1)
                pc = next
            else:
                pc = enext
    return(nth(d,0))

例３にプログラムにおいて，データ d のリストを listnum によって一つの数として表現したもの dd は，次のようにして得られる。

In [10]:
d = [0,5,3]
dd = listnum(d)
print(dd)

3828


mm : プログラム m の，コード（という数）

また，リスト m に対して，m の要素それぞれに f を適用してできたリストは，
```
list (map (f, m))
```
で得られる。よって，m の各要素に listnum を適用て得られるリストは，次のようにして計算できる。

In [11]:
m = [[2,1,1,2],
 [1,0,0],
 [2,2,3,4],
 [1,0,2],
 [0]
]  
m0 = list (map(listnum, m))
print(m0)

[32638, 14, 26364689, 152, 1]


このリストに対して，もう一度 listnum を適用することにより，
プログラム m のコードとなる数が得られる。

In [12]:
mm = listnum (m0)
print(mm)


1830392212574384253594576266619768224916214664365824640643


mm と dd を引数にして univm を呼び出すと，5 + 3 が計算される。

In [14]:
univm(mm, dd)

0 [0, 5, 3]
1 [0, 4, 3]
0 [1, 4, 3]
1 [1, 3, 3]
0 [2, 3, 3]
1 [2, 2, 3]
0 [3, 2, 3]
1 [3, 1, 3]
0 [4, 1, 3]
1 [4, 0, 3]
0 [5, 0, 3]
2 [5, 0, 3]
3 [5, 0, 2]
2 [6, 0, 2]
3 [6, 0, 1]
2 [7, 0, 1]
3 [7, 0, 0]
2 [8, 0, 0]


8

**練習問題1** この下に，次のセルを作成しながら，引き算のプログラムに対して例３と同様のことを行おう。

1. 引き算のプログラムを，Abacus マシンで作成しよう。 (つまり，マシンが停止した時
に，B[0] の値が B[1] - B[2] となっているマシン。最終的に，B[1], B[2] の値は変化していてもよい。）
ただし，ここでの引き算は，x < y の時， x - y = 0 とした引き算である。
その絵を何らかのツールで描いたり，あるいは写真にとって，.jpg ファイルにして取り込むなどして，この notebook に取り込もう。

2. そのプログラムを，リスト表現になおし，listnum を用いて，そのマシンコードとなる数を求めよう。

3. 適当な引数 (例えば，上と同じ，5, 3 でもよい)　に対して univm で動作させ，結果が正しいことを確認しよう。

**練習問題2** 適当な関数を考えて，それに対して，例３と同様のことを行おう。

**練習問題3** この univm では，大きなプログラムを実行することは不可能である。大きなプログラムを実行するには，Abacus マシンのコードの定義を変える必要があろう。どのように定義したら，より大きなプログラムも実行できるようになるか考えよう。